In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url of the website's ranking database

In [ ]:
SITE = "https://www.timeshighereducation.com"
URL = SITE+"/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json"

we access the database and look what is inside

In [ ]:
N = 200
r = requests.get(URL)
all_data = r.json()
print(list(all_data.keys()))
data = all_data.get("data")[:N]
print(list(data[0].keys()))
names = [u.get("name") for u in all_data.get("data")]
print("Extracted %d names" % len(names))

We want to see which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students. And we want to also do it by aggregating by country and region. To do that we observe that both information are already present in the database (already computed in "stats_pc_intl_students", and "stats_student_staff_ratio")

In [ ]:
data[37]

for now not needed -> remove if still the case later

In [ ]:
s = requests.Session()
reqs = [(req_id, s.prepare_request(requests.Request('GET', SITE+entry["url"]))) for req_id, entry in enumerate(data)]
print(len(reqs), "requests to be sent.")

resps = [(req_id, s.send(req)) for req_id, req in reqs]

done = [(req_id, resp.text) for req_id, resp in resps if resp.status_code == 200]
failed = [(req_id, resp) for req_id, resp in resps if resp.status_code != 200]

print("%d done, %d failed." % (len(done), len(failed)))

We clean the dataframe by first renaming the columns:

In [ ]:
uni = pd.DataFrame(data, columns=["name", "rank", "location", "stats_pc_intl_students", "stats_student_staff_ratio", "url"])
uni.rename(columns={"location":"country","stats_pc_intl_students":"pc_intl_students", "stats_student_staff_ratio":"student_staff_ratio"},inplace=True)

Then, we format both ranks and ratios as numbers

In [ ]:
uni["rank"] = uni["rank"].str.extract('(\d+)', expand=False).astype(int)
uni["pc_intl_students"]= uni["pc_intl_students"].str.extract('(\d+)', expand=False).astype(int)

and check that the database has all the values

In [ ]:
uni.isnull().values.any()

save the dataframe in a file

In [ ]:
import pickle

uni.to_pickle("site2.pkl")
pickle.dump(names, open('site2_names.pkl', 'wb'))

In [ ]:
uni.head()